In [1]:
import os
os.environ['CURL_CA_BUNDLE'] = ''

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")

In [1]:
import pandas as pd

In [2]:
data = pd.read_json('dev-v2.0.json')

In [3]:
data

,version,data
0,v2.0,"{'title': 'Normans', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Computational_complexity_theory', '..."
2,v2.0,"{'title': 'Southern_California', 'paragraphs':..."
3,v2.0,"{'title': 'Sky_(United_Kingdom)', 'paragraphs'..."
4,v2.0,"{'title': 'Victoria_(Australia)', 'paragraphs'..."
5,v2.0,"{'title': 'Huguenot', 'paragraphs': [{'qas': [..."
6,v2.0,"{'title': 'Steam_engine', 'paragraphs': [{'qas..."
7,v2.0,"{'title': 'Oxygen', 'paragraphs': [{'qas': [{'..."
8,v2.0,"{'title': '1973_oil_crisis', 'paragraphs': [{'..."
9,v2.0,"{'title': 'European_Union_law', 'paragraphs': ..."


In [6]:
import json
import pandas as pd
# Specify the path to the JSON file
json_file_path = 'dev-v2.0.json'

# Open the JSON file
with open(json_file_path, "r", encoding="utf-8") as f:
    squad_data = json.load(f)


In [7]:
first_article = squad_data["data"][0]
context = first_article["paragraphs"][0]["context"]
# print(context)

In [8]:
squad_data["data"][0]

{'title': 'Normans',
 'paragraphs': [{'qas': [{'question': 'In what country is Normandy located?',
     'id': '56ddde6b9a695914005b9628',
     'answers': [{'text': 'France', 'answer_start': 159},
      {'text': 'France', 'answer_start': 159},
      {'text': 'France', 'answer_start': 159},
      {'text': 'France', 'answer_start': 159}],
     'is_impossible': False},
    {'question': 'When were the Normans in Normandy?',
     'id': '56ddde6b9a695914005b9629',
     'answers': [{'text': '10th and 11th centuries', 'answer_start': 94},
      {'text': 'in the 10th and 11th centuries', 'answer_start': 87},
      {'text': '10th and 11th centuries', 'answer_start': 94},
      {'text': '10th and 11th centuries', 'answer_start': 94}],
     'is_impossible': False},
    {'question': 'From which countries did the Norse originate?',
     'id': '56ddde6b9a695914005b962a',
     'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
      {'text': 'Denmark, Iceland and Norway', 'answer

In [9]:
contexts = []
questions = []
answers = []

# Iterate through the JSON structure and extract context, questions, and the first answer
for article in squad_data["data"]:
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            question = qa["question"]
            answer = qa["answers"][0]["text"] if qa["answers"] else ''  # Extract the first answer, or None if no answers
            contexts.append(context)
            questions.append(question)
            answers.append(answer)

# Create a pandas DataFrame
df = pd.DataFrame({"context": contexts, "question": questions, "answer": answers})

In [10]:
df = df[df['answer'] != '']

In [22]:
contexts = list(df['context'])
questions = list(df['question'])
answers = list(df['answer'])

In [23]:
len(contexts)

5928

In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, RepeatVector
import numpy as np

In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(contexts + questions)
vocab_size = len(tokenizer.word_index) + 1

In [26]:
encoded_contexts = tokenizer.texts_to_sequences(contexts)
encoded_questions = tokenizer.texts_to_sequences(questions)
encoded_answers = tokenizer.texts_to_sequences(answers)

In [27]:
max_length = max(max(len(doc), len(q)) for doc, q in zip(encoded_contexts, encoded_questions))
padded_contexts = pad_sequences(encoded_contexts, maxlen=max_length, padding='post')
padded_questions = pad_sequences(encoded_questions, maxlen=max_length, padding='post')
padded_answers = pad_sequences(encoded_answers, maxlen=max_length, padding='post')

In [28]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_length))
model.add(LSTM(100))
model.add(RepeatVector(max_length))
model.add(LSTM(100, return_sequences=True))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [29]:
model.fit(padded_contexts, np.expand_dims(padded_questions, -1), epochs=5)

Epoch 1/5
  1/186 [..............................] - ETA: 10:59 - loss: 9.8053

KeyboardInterrupt: 

In [32]:
new_document = 'The pound-force has a metric counterpart, less commonly used than the newton bicycle. Other arcane units of force include the sthène, which is equivalent to 1000 N, and the kip, which is equivalent to 1000 lbf.'
encoded_new_document = tokenizer.texts_to_sequences([new_document])
padded_new_document = pad_sequences(encoded_new_document, maxlen=max_length, padding='post')

predicted_questions = model.predict(padded_new_document)
predicted_questions_text = tokenizer.sequences_to_texts(np.argmax(predicted_questions, axis=2))

print(predicted_questions_text[0] + '?')

1/1 [==============================] - 0s 96ms/step
what is the the the?


In [33]:
max_length = max(max(len(ctx), len(q)) for ctx, q in zip(encoded_contexts, encoded_questions))
X = np.concatenate((padded_contexts, padded_questions), axis=1)
Y = tokenizer.texts_to_matrix(answers)

In [40]:
Y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
model1 = Sequential()
model1.add(Embedding(vocab_size, 50, input_length=max_length*2))
model1.add(LSTM(128))  # Adjust LSTM units as needed
model1.add(Dense(Y.shape[1], activation='softmax'))

# Compile the model
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [38]:
model1.fit(X, Y, epochs=5)

Epoch 1/5


TypeError: 'NoneType' object is not callable

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# Example dataset
contexts = ["The quick brown fox jumps over the lazy dog.",
            "Python is a popular programming language.",
            "Deep learning has revolutionized the field of artificial intelligence."]
questions = ["What jumps over the lazy dog?",
             "What is a popular programming language?",
             "What has revolutionized the field of artificial intelligence?"]
answers = ["The quick brown fox",
           "Python",
           "Deep learning"]

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(contexts + questions)
vocab_size = len(tokenizer.word_index) + 1

# Encode contexts and questions
encoded_contexts = tokenizer.texts_to_sequences(contexts)
encoded_questions = tokenizer.texts_to_sequences(questions)

# Pad sequences to ensure equal length
max_length = max(max(len(ctx), len(q)) for ctx, q in zip(encoded_contexts, encoded_questions))
padded_contexts = pad_sequences(encoded_contexts, maxlen=max_length, padding='post')
padded_questions = pad_sequences(encoded_questions, maxlen=max_length, padding='post')

# Prepare input-output pairs
X = np.concatenate((padded_contexts, padded_questions), axis=1)
Y = tokenizer.texts_to_matrix(answers)

# Define LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length*2))
model.add(LSTM(100))
model.add(Dense(Y.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=100, verbose=1)

# Evaluate the model (optional)
# loss, accuracy = model.evaluate(X, Y, verbose=0)
# print('Accuracy: %f' % (accuracy*100))
# Preprocess the input question
input_question = "What jumps over the lazy dog?"
encoded_question = tokenizer.texts_to_sequences([input_question])
padded_question = pad_sequences(encoded_question, maxlen=max_length, padding='post')

# Predict the answer
predicted_answer_probs = model.predict(padded_question)
predicted_answer_index = np.argmax(predicted_answer_probs)
predicted_answer = tokenizer.index_word[predicted_answer_index]

print("Predicted Answer:", predicted_answer)



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, RepeatVector
import numpy as np

# Example dataset (pairs of documents and corresponding questions)
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Python is a popular programming language.",
    "Deep learning has revolutionized the field of artificial intelligence."
]
questions = [
    "What jumps over the lazy dog?",
    "What is a popular programming language?",
    "What has revolutionized the field of artificial intelligence?"
]

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(documents + questions)
vocab_size = len(tokenizer.word_index) + 1

# Encode documents and questions
encoded_documents = tokenizer.texts_to_sequences(documents)
encoded_questions = tokenizer.texts_to_sequences(questions)

# Pad sequences to ensure equal length
max_length = max(max(len(doc), len(q)) for doc, q in zip(encoded_documents, encoded_questions))
padded_documents = pad_sequences(encoded_documents, maxlen=max_length, padding='post')
padded_questions = pad_sequences(encoded_questions, maxlen=max_length, padding='post')

# LSTM Model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_length))
model.add(LSTM(100))
model.add(RepeatVector(max_length))
model.add(LSTM(100, return_sequences=True))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model
model.fit(padded_documents, np.expand_dims(padded_questions, -1), epochs=100, verbose=0)

# Generate questions from a new document
new_document = "The cat sat on the mat."
encoded_new_document = tokenizer.texts_to_sequences([new_document])
padded_new_document = pad_sequences(encoded_new_document, maxlen=max_length, padding='post')

predicted_questions = model.predict(padded_new_document)
predicted_questions_text = tokenizer.sequences_to_texts(np.argmax(predicted_questions, axis=2))

print(predicted_questions_text)
